# Studying qNML for Bayesian networks

NOTICE! This notebook requires networkx, scipy, and numpy

## Recap of definitions of NML and qNML


$$
nml(D;G) = \frac{P(D;\hat\theta_G(D))}{\sum_{D'\sim D} P(D';\hat\theta_G(D'))},
$$ 

where $\theta_G(D)$ denotes the maximum likelihood paramaters for the data $D$ in the graph $G$.

For qNML we decompose the graph with $m$ nodes into $m$ parent sets $G=(G_1, \ldots, G_m)$, and further into families $F=(F_1, \ldots, F_m)$, where $F_i = G_i \cup \{i\}$. We select the attributes $S$ of the data sets by denoting $D_{S}$.

$$
qnml(D;G) = \prod_{i=1}^m \frac{nml_1(D_{F_i})}  {nml_1(D_{G_i})},
$$

where we have used a notation $nml_1(D_S)$ to denote NML for a dataset where $D_S$ is collapsed to a one dimensional dataset of size $N$ with $\prod_{i \in S} r_i$ possible discrete values.

## A hypothesis

In particular we study the hypothesis that qNML for a given structure never both over- and underestimates NML for all data formats and sizes:
Denoting that two data sets $D$ and $D'$ have same format and size by $D \sim D'$, we hypothesize that:

$$
\forall G\;  \forall D, D'\;:  (D \sim D') \implies (nml(D;G) \gt qnml(D;G) \implies nml(D';G) \ge qnml(D';G)) 
$$ and 
$$
\forall G\;  \forall D, D'\;:  (D \sim D') \implies (nml(D;G) \lt qnml(D;G) \implies nml(D';G) \le qnml(D';G)). 
$$

If true, this means that qNML induced error in estimation of NML is a structural property of a graph $G$ and we could find a correction term for a given $G$ such that makes the result closer to the real NML.

### NOTICE 

From the equation (10) and the Lemma 1 of the original qNML paper, we can easily see that the difference between NML and qNML, for any given structure $G$, is constant for all datasets that share the same format (i.e., size N and the number of discrete values per variable).
That for a given $G$ this difference is always to the same direction regardless of the data format is currently a mystery to me. 

### Generating all possible datasets

The startegy is to generate all possible datavectors and then use integer partions for data size $N$ to generate all possible datasets. Integer partitions are not enough because it does not contain zeros. If we have $Q$ possible data vectors, and if $N$ is divided to $K$ parts, we have to go through all $Q \choose K$ possible combinations of vectors. 

In [44]:
from itertools import product, combinations
from sympy.utilities.iterables import partitions

def gen_possible_data_vectors(rs):
    return product(*map(range,rs))

def gen_1(partitems, vixset):
    if len(partitems) == 0: 
        yield []
        return
    items = partitems[:]
    (count, mult) = items.pop(0)
    for vs in combinations(vixset, mult):
        vec0 = [(v,count) for v in vs]
        for vec_rest in gen_1(items, vixset - set(vs)):
            yield vec0 + vec_rest
    
def gen_datasets(vectors, N):
    Q = len(vectors)
    vixs = range(Q)
    for partition in partitions(N, m=Q):
        # print(partition)
        partitems = list(partition.items())
        
        counts = sum(([k]*n for k,n in partition.items()), [])
        # print('counts = ', counts)
        K = len(counts)
        for vixset in combinations(vixs, K):
            # print('c',vixset)
            for vec in gen_1(partitems, set(vixset)):
                # print(' ', vec)
                yield(vec)
            # print()

But this will just tell how many vectors of different kinds there are in the dataset. They can still be in different orders number of which is given by a multinomial coeffient. 

In [2]:
from scipy.special import binom
import numpy as np
from functools import lru_cache
@lru_cache
def mulco(ks): 
    ns = np.cumsum(ks)
    binoms = binom(ns,ks)
    return np.prod(binoms)

def mulco_D(D):
    return mulco(tuple(count for _vix,count in D))

Just checking that the counts are right

In [ ]:
# %%time
total_count = 0
N = 7
rs = [2, 3, 2]
vectors = list(gen_possible_data_vectors(rs))
for D in gen_datasets(vectors, N):
    total_count += mulco_D(D)
print(total_count, np.prod(rs)**N)

35831808.0 35831808


## For generating graphs let us just use networkx

But we still need to gather statistics to find maximum likelihoods. For that we have to extract the counts per parent configurations

In [ ]:
from collections import defaultdict

def gen_parents(g):
    for n in g:
        yield (n, g.predecessors(n))

def gen_counts(vectors, D, g):
    for (n,ps) in gen_parents(g): # go through nodes and their parents
        pset = frozenset(ps)
        for vix, count in D:      # go through the data
            d = vectors[vix]
            v = d[n]
            pcfg = frozenset((p,d[p]) for p in pset)
            yield ((n,v), pcfg, count)

def collect_counts(vectors, D, g): # could take N and rs
    N_vps = [defaultdict(lambda:defaultdict(int)) for _n in g]
    for ((n,v), pcfg, c) in gen_counts(vectors, D, g):
        N_vps[n][pcfg][v] += c
    return N_vps


Bit of testing

In [5]:
N, rs

(7, [2, 3, 2])

In [6]:
from collections import Counter
def random_data(vectors, N):
    rng = np.random.default_rng()
    rints = rng.integers(low=0, high=len(vectors), size=N)
    return list(Counter(rints).items())                         

In [ ]:
import networkx as nx

g = nx.DiGraph()
g.add_edges_from([(0,2), (1,2)])

vectors = list(gen_possible_data_vectors(rs))
D = random_data(vectors, N)

N_vps = collect_counts(vectors, D, g)

print(D)

for (vix,c) in D:
    print(vectors[vix], c)
print()

for n,N_vp in enumerate(N_vps):
    print('node',n, sorted(g.predecessors(n)))
    for pcfg, counts in sorted(N_vp.items()):
        print(f'N({[v for (p,v) in sorted(pcfg)]}) = {sorted(counts.items())}')


[(np.int64(0), 1), (np.int64(10), 2), (np.int64(6), 1), (np.int64(7), 1), (np.int64(1), 1), (np.int64(4), 1)]
(0, 0, 0) 1
(1, 2, 0) 2
(1, 0, 0) 1
(1, 0, 1) 1
(0, 0, 1) 1
(0, 2, 0) 1

node 0 []
N([]) = [(0, 3), (1, 4)]
node 1 []
N([]) = [(0, 4), (2, 3)]
node 2 [0, 1]
N([0, 0]) = [(0, 1), (1, 1)]
N([1, 2]) = [(0, 2)]
N([1, 0]) = [(0, 1), (1, 1)]
N([0, 2]) = [(0, 1)]


Looks right !!

### So now we could get maximum likelihoods

In [8]:
from scipy.stats import entropy

def log_ml(N_vps):
    res  = 0.0
    for N_vp in N_vps:
        for counts in N_vp.values():
            freqs = np.array(list(counts.values()))
            res -= freqs.sum() * entropy(freqs)
    return res

In [9]:
log_ml(N_vps)

np.float64(-12.333302188046385)

### and NML distribution

In [10]:
def gen_log_mls(vectors, N, g):
    for D in gen_datasets(vectors, N):
        N_vps = collect_counts(vectors, D, g)
        yield (log_ml(N_vps), mulco_D(D))

In [11]:
from pprint import pprint

def log_mls_counts(vectors, N, g):
    log_mls = defaultdict(int)
    for lml, c in gen_log_mls(vectors, 4, g):
        log_mls[lml] += c
    return log_mls

pprint(log_mls_counts(vectors, 2, g))

defaultdict(<class 'int'>,
            {np.float64(-8.317766166719343): np.float64(720.0),
             np.float64(-7.7945180229547955): np.float64(1152.0),
             np.float64(-6.931471805599453): np.float64(288.0),
             np.float64(-6.931471805599452): np.float64(2880.0),
             np.float64(-6.408223661834905): np.float64(6336.0),
             np.float64(-5.884975518070357): np.float64(1152.0),
             np.float64(-5.545177444479562): np.float64(2088.0),
             np.float64(-5.021929300715014): np.float64(2304.0),
             np.float64(-4.498681156950466): np.float64(1344.0),
             np.float64(-4.1588830833596715): np.float64(1872.0),
             np.float64(-2.772588722239781): np.float64(252.0),
             np.float64(-2.249340578475233): np.float64(336.0),
             np.float64(0.0): np.float64(12.0)})


In [12]:
def log_nml_denom(vectors, N):
    denom = 0
    for lml,c in gen_log_mls(vectors, N, g):
        denom += np.exp(lml) * c
    return np.log(denom)

## qNML

In [13]:
from itertools import chain
from nml import lognml
def log_qnml1(freqs, q):    
    freqs0 = [0]*(q-len(freqs)) + freqs
    return lognml(freqs0)

In [14]:
def log_qnml(N_vps, rs, g):
    qs = [int(np.prod([rs[p] for p in g.predecessors(n)])) for n in range(len(g))]
    res = 0
    for N_vp, r, q in zip(N_vps, rs, qs):
        freqs_f = list(chain(*(counts.values() for counts in N_vp.values())))
        freqs_p = list(sum(counts.values()) for counts in N_vp.values())
        res += log_qnml1(freqs_f, q*r) 
        res -= log_qnml1(freqs_p,q)
    return res

## Empirical study of our hypothesis

In [49]:
def get_min_n_max_diffs(vectors, N, g):
    
    def gen_diffs():
        log_denom = log_nml_denom(vectors, N)
        for D in gen_datasets(vectors, N):
            N_vps = collect_counts(vectors, D, g)
            log_nml = log_ml(N_vps) - log_denom
            yield log_qnml(N_vps, rs, g) - log_nml
            
    difflist = np.array(list(gen_diffs()), dtype='float')
    return np.min(difflist), np.max(difflist)


In [16]:
def get_rss(n, max_r):
    rrange = range(2,max_r+1)
    return list(product(rrange, repeat=n))

In [17]:
def get_graph(*nodges):
    g = nx.DiGraph()
    for nodge in nodges:
        if isinstance(nodge, int):
            g.add_node(nodge)
        else:
            g.add_edge(*nodge)
    return g

### Try some graphs where there should be no difference

In [ ]:
from random import choice

graphs = [
    get_graph((0,1), (2,3)),
    get_graph(0, (1,2), (1,3), (2,3)),
    get_graph(0,(1,2),3),
    # get_graph((0,1),(0,2),(0,3),(1,2),(1,3),(2,3))
]

for g in graphs:
    rss = get_rss(len(g.nodes()),3)
    rs = choice(rss)
    vectors = list(gen_possible_data_vectors(rs))    
    N = choice(list(range(2,5)))
    d = get_min_n_max_diffs(vectors, N, g)
    print(N, rs, d)
    

2 (3, 3, 2, 3) (np.float64(-8.881784197001252e-16), np.float64(1.7763568394002505e-15))
2 (3, 2, 2, 2) (np.float64(0.0), np.float64(8.881784197001252e-16))
4 (2, 2, 2, 3) (np.float64(-3.552713678800501e-15), np.float64(1.7763568394002505e-15))


### Try some graphs where there should be difference

In [ ]:
graphs = [
    get_graph((0,1), (0,2)), # A
    get_graph((0,1), (1,2)), # I
    get_graph((0,1), (1,2), (2,3)), # long I
    get_graph((0,2), (1,2)), # V
 #    get_graph((0,2), (1,2), (2,3)), # Y
 #    get_graph((0,1), (1,2), (1,3)), # Y'
 #    get_graph((0,2), (1,2), (2,4), (3,4)), # W'
 #    get_graph((0,1), (1,2), (1,3), (1,3)), # Y+
 #    get_graph((0,2), (1,2), (2,3), (2,4)), # X
 #    get_graph((0,3), (1,3), (2,3), (2,4), (2,5)), # X^
]

for g in graphs:
    rss = get_rss(len(g.nodes()),3)
    rs = choice(rss)
    vectors = list(gen_possible_data_vectors(rs))    
    N = choice(list(range(2,3)))
    mindiff, maxdiff = get_min_n_max_diffs(vectors, N, g)
    print(N, rs, f'{mindiff:.5f} - {maxdiff:.5f}')
    

gc (0, 0, 0) 0
gc (0, 0, 0) 1
gc (0, 0, 0) 2
gc (0, 0, 1) 0
gc (0, 0, 1) 1
gc (0, 0, 1) 2
gc (0, 1, 0) 0
gc (0, 1, 0) 1
gc (0, 1, 0) 2
gc (0, 1, 1) 0
gc (0, 1, 1) 1
gc (0, 1, 1) 2
gc (1, 0, 0) 0
gc (1, 0, 0) 1
gc (1, 0, 0) 2
gc (1, 0, 1) 0
gc (1, 0, 1) 1
gc (1, 0, 1) 2
gc (1, 1, 0) 0
gc (1, 1, 0) 1
gc (1, 1, 0) 2
gc (1, 1, 1) 0
gc (1, 1, 1) 1
gc (1, 1, 1) 2
gc (0, 0, 0) 0
gc (0, 0, 1) 0
gc (0, 0, 0) 1
gc (0, 0, 1) 1
gc (0, 0, 0) 2
gc (0, 0, 1) 2
gc (0, 0, 0) 0
gc (0, 1, 0) 0
gc (0, 0, 0) 1
gc (0, 1, 0) 1
gc (0, 0, 0) 2
gc (0, 1, 0) 2
gc (0, 0, 0) 0
gc (0, 1, 1) 0
gc (0, 0, 0) 1
gc (0, 1, 1) 1
gc (0, 0, 0) 2
gc (0, 1, 1) 2
gc (0, 0, 0) 0
gc (1, 0, 0) 0
gc (0, 0, 0) 1
gc (1, 0, 0) 1
gc (0, 0, 0) 2
gc (1, 0, 0) 2
gc (0, 0, 0) 0
gc (1, 0, 1) 0
gc (0, 0, 0) 1
gc (1, 0, 1) 1
gc (0, 0, 0) 2
gc (1, 0, 1) 2
gc (0, 0, 0) 0
gc (1, 1, 0) 0
gc (0, 0, 0) 1
gc (1, 1, 0) 1
gc (0, 0, 0) 2
gc (1, 1, 0) 2
gc (0, 0, 0) 0
gc (1, 1, 1) 0
gc (0, 0, 0) 1
gc (1, 1, 1) 1
gc (0, 0, 0) 2
gc (1, 1, 1) 2
gc (0, 0, 

Running the cell above many times it looks like our hypothesis is true, V and W networks tend to underestimate the NML and the rest overestimate it. This suggests a bias against converging structures when selecting the model by qNML - this serves as a kind of additional complexity penalty 

In [50]:
import matplotlib.pyplot as plt

# Plot qNML - NML difference for graph g and rs as a function of sample size N
def plot_diff(g, rs, max_N):
    vectors = list(gen_possible_data_vectors(rs))
    N_range = range(1, max_N)
    diffs = []
    for N in N_range:
        min_diff, max_diff = get_min_n_max_diffs(vectors, N, g)
        diffs.append((min_diff, max_diff))

    min_diffs, max_diffs = zip(*diffs)
    plt.plot(N_range, min_diffs, label='min(qNML-NML)')
    plt.plot(N_range, max_diffs, label='max(qNML-NML)')
    plt.xlabel('Sample size N')
    plt.ylabel('qNML - NML')
    plt.title('qNML - NML difference vs. sample size N')
    plt.legend()
    plt.show()
 

In [ ]:
g = get_graph((0,2), (1,2))
rs = (2,2,2)
max_N = 8
plot_diff(g, rs, max_N)


gc (0, 0, 0) 0
gc (0, 0, 0) 2
gc (0, 0, 0) 1
gc (0, 0, 1) 0
gc (0, 0, 1) 2
gc (0, 0, 1) 1
gc (0, 1, 0) 0
gc (0, 1, 0) 2
gc (0, 1, 0) 1
gc (0, 1, 1) 0
gc (0, 1, 1) 2
gc (0, 1, 1) 1
gc (1, 0, 0) 0
gc (1, 0, 0) 2
gc (1, 0, 0) 1
gc (1, 0, 1) 0
gc (1, 0, 1) 2
gc (1, 0, 1) 1
gc (1, 1, 0) 0
gc (1, 1, 0) 2
gc (1, 1, 0) 1
gc (1, 1, 1) 0
gc (1, 1, 1) 2
gc (1, 1, 1) 1
gc (0, 0, 0) 0
gc (0, 0, 0) 2
gc (0, 0, 0) 1
gc (0, 0, 1) 0
gc (0, 0, 1) 2
gc (0, 0, 1) 1
gc (0, 1, 0) 0
gc (0, 1, 0) 2
gc (0, 1, 0) 1
gc (0, 1, 1) 0
gc (0, 1, 1) 2
gc (0, 1, 1) 1
gc (1, 0, 0) 0
gc (1, 0, 0) 2
gc (1, 0, 0) 1
gc (1, 0, 1) 0
gc (1, 0, 1) 2
gc (1, 0, 1) 1
gc (1, 1, 0) 0
gc (1, 1, 0) 2
gc (1, 1, 0) 1
gc (1, 1, 1) 0
gc (1, 1, 1) 2
gc (1, 1, 1) 1
gc (0, 0, 0) 0
gc (0, 0, 0) 2
gc (0, 0, 0) 1
gc (0, 0, 1) 0
gc (0, 0, 1) 2
gc (0, 0, 1) 1
gc (0, 1, 0) 0
gc (0, 1, 0) 2
gc (0, 1, 0) 1
gc (0, 1, 1) 0
gc (0, 1, 1) 2
gc (0, 1, 1) 1
gc (1, 0, 0) 0
gc (1, 0, 0) 2
gc (1, 0, 0) 1
gc (1, 0, 1) 0
gc (1, 0, 1) 2
gc (1, 0, 1) 1
gc (1, 1, 